##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API：REST 快速入門嵌入

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/quickstarts/rest/Embeddings_REST.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


本筆記本提供快速的程式碼範例，說明如何使用 `curl` 開始生成內嵌碼。

你可以在 Google Colab 中執行此程式碼，或者將 `curl` 指令複製/貼進你的終端命令。

若要執行本筆記本，你的 API 金鑰必須儲存在稱為 GOOGLE_API_KEY 的 Google 秘鑰中。如果你在其他環境中執行，你可以將金鑰儲存在環境變數中。請參閱 [驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 以進一步了解。


In [1]:
import os
from google.colab import userdata

In [2]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

## 嵌入內容

呼叫具備 `text-embedding-004` 模型的 `embed_content` 方法，以產生文字嵌入：


In [3]:
%%bash

curl "https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?key=$GOOGLE_API_KEY" \
-H 'Content-Type: application/json' \
-d '{"model": "models/text-embedding-004",
    "content": {
    "parts":[{
      "text": "Hello world"}]}, }' 2> /dev/null | head

{
  "embedding": {
    "values": [
      0.013168523,
      -0.008711934,
      -0.046782676,
      0.00069968984,
      -0.009518873,
      -0.008720178,
      0.060103577,


# 批次嵌入內容

你可以使用單一 API 呼叫來嵌入多個提示列表以提高效率。


In [4]:
%%bash

curl "https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents?key=$GOOGLE_API_KEY" \
-H 'Content-Type: application/json' \
-d '{"requests": [{
      "model": "models/text-embedding-004",
      "content": {
      "parts":[{
        "text": "What is the meaning of life?"}]}, },
      {
      "model": "models/text-embedding-004",
      "content": {
      "parts":[{
        "text": "How much wood would a woodchuck chuck?"}]}, },
      {
      "model": "models/text-embedding-004",
      "content": {
      "parts":[{
        "text": "How does the brain work?"}]}, }, ]}' 2> /dev/null | grep -C 5 values

{
  "embeddings": [
    {
      "values": [
        -0.010632277,
        0.019375855,
        0.0209652,
        0.0007706424,
        -0.061464064,
--
        -0.0071538696,
        -0.028534694
      ]
    },
    {
      "values": [
        0.018467998,
        0.0054281196,
        -0.017658804,
        0.013859266,
        0.053418662,
--
        0.026714385,
        0.0018762538
      ]
    },
    {
      "values": [
        0.05808907,
        0.020941721,
        -0.108728774,
        -0.04039259,
        -0.04440443,


## 設定輸出維度
如果你使用的是 `text-embeddings-004`，你可以設定 `output_dimensionality` 參數來建立更小的嵌入。

* `output_dimensionality` 會截斷嵌入 (例如，當 `output_dimensionality=2` 時，`[1, 3, 5]` 會變成 `[1,3]` )。


In [5]:
%%bash

curl "https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?key=$GOOGLE_API_KEY" \
-H 'Content-Type: application/json' \
-d '{"model": "models/text-embedding-004",
    "output_dimensionality":256,
    "content": {
    "parts":[{
      "text": "Hello world"}]}, }' 2> /dev/null | head

{
  "embedding": {
    "values": [
      0.013168523,
      -0.008711934,
      -0.046782676,
      0.00069968984,
      -0.009518873,
      -0.008720178,
      0.060103577,


## 使用 `task_type` 模型來提供提示，說明你將如何使用嵌入

讓我們看看嵌入方法所採取的所有參數。有四個：

* `model`：必填。必須為 `models/embedding-001`。
* `content`：必填。你想要嵌入的內容。
* `task_type`：選填。將使用嵌入的任務類型。請見以下可能的數值。
* `title`：給定的文字是正在搜尋的語料庫的檔案。選擇性地設定 `title` 參數，並輸入該檔案的標題。只能在 `task_type` 為 `RETRIEVAL_DOCUMENT` 時設定。

`task_type` 是選填參數，針對你打算在應用程式中如何使用嵌入，提供給 API 的提示。

接受以下 task_type 參數：

* `TASK_TYPE_UNSPECIFIED`：如果你未設定值，會預設為 `retrieval_query`。
* `RETRIEVAL_QUERY`：給定的文字是搜尋/檢索設定中的查詢。
* `RETRIEVAL_DOCUMENT`：給定的文字是正在搜尋的語料庫的檔案。
* `SEMANTIC_SIMILARITY`：給定的文字將用於語意文字相似性 (STS)。
* `CLASSIFICATION`：將對給定的文字進行分類。
* `CLUSTERING`：嵌入將用於分群。


In [6]:
%%bash

curl "https://generativelanguage.googleapis.com/v1beta/models/embedding-001:embedContent?key=$GOOGLE_API_KEY" \
-H 'Content-Type: application/json' \
-d '{"model": "models/text-embedding-004",
    "content": {
    "parts":[{
      "text": "Hello world"}]},
    "task_type": "RETRIEVAL_DOCUMENT",
    "title": "My title"}' 2> /dev/null | head

{
  "embedding": {
    "values": [
      0.060187872,
      -0.031515103,
      -0.03244149,
      -0.019341845,
      0.057285223,
      0.037159503,
      0.035636507,


## 了解更多資訊

* 在 [此處](https://developers.googleblog.com/2024/04/gemini-15-pro-in-public-preview-with-new-features.html)進一步了解 text-embeddings-004。
*   看看 [REST API 參考](https://ai.google.dev/api/rest) 以進一步了解更多資訊。
*   在食譜中探索更多範例。
